In [ ]:
from PIL import Image
import pytesseract
import cv2
import os
import json
import re
import string
%pylab inline

In [ ]:
def cleanup(text):
    text = text.replace('\n',' ')
    tokens = text.split(' ')
    new_tokens = []
    for token in tokens:
        token = token.strip()
        token = re.sub(r'[^\w\s]', ' ', token)
        token = ''.join(e for e in token if e.isalpha())
        new_tokens.append(token)
    return ' '.join(new_tokens)

def text_extractor(file_name):
    img = cv2.imread(file_name)
    img_final = img.copy()
    img2gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(img2gray, 127, 255, cv2.THRESH_BINARY)
    new_img = cv2.bitwise_and(img2gray, img2gray, mask=mask)
    ret, new_img = cv2.threshold(new_img, 127, 255, cv2.THRESH_BINARY)  # for black text , cv.THRESH_BINARY_INV
    '''
            line  8 to 12  : Remove noisy portion 
    '''
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,
                                                         3))  # to manipulate the orientation of dilution , large x means horizonatally dilating  more, large y means vertically dilating more
    dilated = cv2.dilate(new_img, kernel, iterations=9)  # dilate , more the iteration more the dilation

    # for cv2.x.x

    #_, contours, hierarchy = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)  # findContours returns 3 variables for getting contours

    # for cv3.x.x comment above line and uncomment line below

    image, contours, hierarchy = cv2.findContours(dilated,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    cropped_imgs = []
    textboxes = []
    for contour in contours:
        # get rectangle bounding contour
        [x, y, w, h] = cv2.boundingRect(contour)

        # Don't plot small false positives that aren't text
        if w < 10 and h < 10:
            continue
            
        # draw rectangle around contour on original image
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 255), 2)

        '''
        #you can crop image and send to OCR  , false detected will return no text :)
        cropped = img_final[y :y +  h , x : x + w]

        s = file_name + '/crop_' + str(index) + '.jpg' 
        cv2.imwrite(s , cropped)
        index = index + 1

        '''
        cropped = img_final[y :y +  h , x : x + w]
        cropped_imgs.append(cropped)
        
        text = pytesseract.image_to_string(cropped)
        if len(text) > 0:
            textboxes.append([text, x, y])
        
    textboxes = sorted(textboxes, key=lambda x: (x[2], x[1]))
    text = ' '.join([text[0] for text in textboxes])
    print(text)
    return cleanup(text)



In [ ]:
text_extractor('./opencv-text-recognition/images/example_04.jpg')

In [ ]:
text = '''Life ls Strange is a story based game that features player choice, the consequences
of all yow in game actions and decisions will impact the past, pvesent and futuve.
Choose wisely“.'''
print(cleanup(text))

In [ ]:
%%time
json_file = open('./data source/Life Is Strange 1/output/lifeisstrange_w_img_keywords.json')
database = json.loads(json_file.read())
for block in database:
    folder = block['screenshots']['image_folder']
    sessions = block['screenshots']['image_info']
    for session in sessions:
        for key, image in session['frames'].items():
            file_path = os.path.join(folder, image['image_file'])
            texts = image_to_texts(file_path)
            print(texts)
            session['frames'][key]['image_texts'] = texts

with open('./data source/Life Is Strange 1/output/lifeisstrange_w_img_texts.json', 'w+') as output_file:
    json.dump(database, output_file, indent=4)